# News Scrapers

In [60]:
import requests
import json 
import pandas as pd
import html2text
import tqdm.notebook as tqdm
import copy

GUARDIAN_API_KEY =  "6c15c612-076e-4fff-9023-194e033a4c85"

## Utility Functions

In [51]:
def print_json(results): 
    print(json.dumps(results, indent=2))

h = html2text.HTML2Text()
h.ignore_links = True

## Guardian

1. Get total number of pages of articles containing the words "coronavirus" and "covid-19"

In [52]:
search_query = 'coronavirus|covid-19'
from_date = '2019-12-25'
order_by = 'oldest'
page_size = '50'
fields = 'headline'
blocks = 'body'

fetch_url = f'https://content.guardianapis.com/search?q={search_query}&from-date={from_date}&order-by={order_by}&page-size={page_size}&show-fields={fields}&show-blocks={blocks}&api-key={GUARDIAN_API_KEY}'

r = requests.get(fetch_url).json()

number_of_pages = r['response']['pages']


2. Fetch every article from every page

In [110]:
content = {}

content_no = 0

for page in tqdm.tqdm(range(1, number_of_pages + 1)):
    fetch_url = f'https://content.guardianapis.com/search?q={search_query}&from-date={from_date}&order-by={order_by}&page-size={page_size}&show-fields={fields}&show-blocks={blocks}&page={page}&api-key={GUARDIAN_API_KEY}'
    r = requests.get(fetch_url).json()
   
    results = r['response']['results']   

    for result in tqdm.tqdm(results, leave=False):
        
        content[content_no] = {
            'id': result['id'],
            'section': result['sectionName'],
            'date_published': result['webPublicationDate'],
            'headline': result['fields']['headline'],
            'url': result['webUrl'],
            'body': h.handle(result['blocks']['body'][0]['bodyHtml'])
        }
        
        content_no += 1

print(f"Number of articles found: {len(content.keys())}")   


Number of articles found: 2187


3. Preprocess the data to remove any articles without the words "coronavirus" or "covid-19"

In [111]:
ids = list(content.keys())

for id in tqdm.tqdm(ids):
    entry = content[id]
    body = entry['body']
    body = body.split()
        
    no_coronavirus = "coronavirus" not in body
    no_covid19 = "covid-19" not in body
    no_COVID19 = "COVID-19" not in body
        
    if no_coronavirus and no_covid19 and no_COVID19:
        content.pop(id)

# Create final dict with correct article no
processed_content = {}
content_no = 0
        
for id in content.keys():
    processed_content[content_no] = content[id]
    content_no += 1

4. Save data as `json`

In [112]:
with open('news.json', 'w') as fp:
    json.dump(processed_content, fp)

In [117]:
df = pd.DataFrame(processed_content)

df.T

1766


,id,section,date_published,headline,url,body
0,sport/2019/dec/31/sport-in-2020-calendar-your-...,Sport,2019-12-31T20:00:44Z,Sport in 2020 calendar: your month-by-month gu...,https://www.theguardian.com/sport/2019/dec/31/...,\n\n * _A number of sporting events have been...
1,world/2020/jan/09/chinas-sars-like-illness-wor...,World news,2020-01-09T19:10:17Z,China's Sars-like illness worries health experts,https://www.theguardian.com/world/2020/jan/09/...,\n\nThe finding that the outbreak of viral pne...
2,world/2020/jan/18/coronavirus-what-airport-mea...,World news,2020-01-18T09:21:23Z,Coronavirus: what airport measures are in plac...,https://www.theguardian.com/world/2020/jan/18/...,\n\nInternational airports are stepping up scr...
3,australia-news/2020/jan/18/coronavirus-austral...,Australia news,2020-01-18T10:29:33Z,Coronavirus: Australia's top health official s...,https://www.theguardian.com/australia-news/202...,\n\nAustralia’s top health official says there...
4,society/2020/jan/18/expert-questions-effective...,Society,2020-01-18T19:20:36Z,Expert questions effectiveness of coronavirus ...,https://www.theguardian.com/society/2020/jan/1...,\n\nMedical staff at airports trying to screen...
...,...,...,...,...,...,...
1761,world/2020/mar/14/trump-coronavirus-travel-ban...,World news,2020-03-14T18:02:25Z,Will Trump's coronavirus travel ban work? Scie...,https://www.theguardian.com/world/2020/mar/14/...,\n\nDonald Trump’s dramatic announcement this ...
1762,world/2020/mar/14/solidarity-balcony-singing-s...,World news,2020-03-14T18:18:15Z,Solidarity balcony singing spreads across Ital...,https://www.theguardian.com/world/2020/mar/14/...,\n\nBalcony singing in solidarity has been a g...
1763,commentisfree/2020/mar/14/coronavirus-in-an-er...,Opinion,2020-03-14T18:30:35Z,The great divide is between the well and the s...,https://www.theguardian.com/commentisfree/2020...,\n\nSpeaking to parliament in 1855 as the carn...
1764,world/2020/mar/14/talk-about-death-coronavirus...,World news,2020-03-14T18:51:00Z,We can’t be squeamish about death. We need to ...,https://www.theguardian.com/world/2020/mar/14/...,\n\nAs the coronavirus spreads through the Bri...
